# Cofactors

In [1]:
import reframed
from reframed import pFBA, Environment, FVA,FBA
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict 


#### Loading model

In [2]:
model = reframed.load_cbmodel('model_cellulolyticum_H10.xml')

#### Gene-protein cross-reference

In [3]:
%store -r gene_protein_map

In [8]:
gene_protein_map.head(3)

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
0,B8I4G1,LEUD_RUMCH,3-isopropylmalate dehydratase small subunit (E...,leuD Ccel_0127,G_WP_012634581_1
1,B8I8F2,UVRC_RUMCH,UvrABC system protein C (Protein UvrC) (Excinu...,uvrC Ccel_0807,G_WP_015924347_1
2,B8I567,UPP_RUMCH,Uracil phosphoribosyltransferase (EC 2.4.2.9) ...,upp Ccel_0260,G_WP_012634712_1


## Ferredoxin and NAD(P)H balancing

**Motivation**: The model is lacking some reactions described in literature that are related to cofactor balancing. This is especially related to ferredoxin reductase activity. 



### Analysis

**Reactions involving ferredoxin that we want to be in the system (based on Desvaux 2001)**
- R_POR_syn: M_coa_c + 2.0 M_fdxo_2_2_c + M_pyr_c --> M_accoa_c + M_co2_c + 2.0 M_fdxrd_c + M_h_c
- R_FNRR: M_fdxrd_c + M_h_c + M_nad_c <-> M_fdxo_2_2_c + M_nadh_c
- R_FNRR2: M_fdxrd_c + M_h_c + M_nadp_c <-> M_fdxo_2_2_c + M_nadph_c
- R_FNRR3: M_fdxo_2_2_c + M_h2_c --> M_fdxrd_c + 2.0 M_h_c


**Reactions involving ferredoxin currently in model**

In [79]:
for rxn in model.get_metabolite_reactions('M_fdxrd_c'):
    print(str(model.reactions[rxn])+ ": "+str(model.reactions[rxn].gpr))

R_CO2FO: M_co2_c + M_fdxrd_c + 2.0 M_h_c <-> M_co_c + M_fdxo_2_2_c + M_h2o_c [-inf, 0.0]: G_WP_015926513_1
R_FNRR3: M_fdxo_2_2_c + M_h2_c --> M_fdxrd_c + 2.0 M_h_c: G_WP_015925729_1
R_MECDPDH4E: M_2mecdp_c + M_fdxrd_c + M_h_c <-> M_fdxo_2_2_c + M_h2mb4p_c + M_h2o_c: G_WP_012634895_1
R_NOR_syn_1: 6.0 M_fdxrd_c + 7.0 M_h_c + M_no2_c --> 6.0 M_fdxo_2_2_c + 2.0 M_h2o_c + M_nh3_c: None
R_POR_syn: M_coa_c + 2.0 M_fdxo_2_2_c + M_pyr_c --> M_accoa_c + M_co2_c + 2.0 M_fdxrd_c + M_h_c: (G_WP_015924110_1 and G_WP_015924674_1)
R_SULR_syn: 6.0 M_fdxrd_c + 8.0 M_h_c + M_so3_c --> 6.0 M_fdxo_2_2_c + 3.0 M_h2o_c + M_h2s_c: None
R_VOR2bE: M_3mob_c + M_coa_c + M_fdxo_2_2_c --> M_co2_c + M_fdxrd_c + M_h_c + M_ibcoa_c: G_WP_015924108_1


*Of these reactions, two do not have a GPR*:

- R_NOR_syn_1: From Synechocystis sp. PCC68 model (iSynCJ816) - involved in [nitrogen metabolism and nitrate assimilation](https://www.brenda-enzymes.org/enzyme.php?ecno=1.7.7.1)
- R_SULR_syn: From Synechocystis sp. PCC68 model (iSynCJ816) - involved in [sulfate assimilation](https://www.brenda-enzymes.org/enzyme.php?ecno=1.8.7.1)

model.remove_reactions(["R_NOR_syn_1","R_SULR_syn"])

**POR_syn - GPRs**

In [80]:
model.reactions.R_POR_syn.gpr

(G_WP_015924110_1 and G_WP_015924674_1)

In [81]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015924110_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
616,B8I6P6,B8I6P6_RUMCH,Pyruvate ferredoxin/flavodoxin oxidoreductase,Ccel_0556,G_WP_015924110_1


In [82]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015924674_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1110,B8I021,B8I021_RUMCH,Pyruvate flavodoxin/ferredoxin oxidoreductase ...,Ccel_1164,G_WP_015924674_1


**FNRR3 - GPRs in model**

<span style="color: red;">Not sure if this one makes sense. </span>

In [83]:
model.reactions.R_FNRR3.gpr

G_WP_015925729_1

In [84]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925729_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
387,B8I4Y3,B8I4Y3_RUMCH,Putative PAS/PAC sensor protein,Ccel_2300,G_WP_015925729_1


### Sugar metabolism

**cellobiose phosphorylase**

In [12]:
model.reactions.R_CEPA

R_CEPA: M_cellb_c + M_pi_c <-> M_g1p_c + M_glc__D_c

In [9]:
model.reactions.R_CEPA.gpr

G_WP_015925554_1

In [10]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925554_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
872,B8I421,B8I421_RUMCH,Glycosyltransferase 36,Ccel_2109,G_WP_015925554_1


In [168]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015925554_1']:
    print(str(model.reactions[rxn])+": " +  str(model.reactions[rxn].gpr))

R_CEPA: M_cellb_c + M_pi_c <-> M_g1p_c + M_glc__D_c: G_WP_015925554_1


**Galactokinase**

In [142]:
model.reactions.R_GALK2

R_GALK2: M_a_gal__D_c + M_atp_c --> M_adp_c + M_gal1p_c + M_h_c

In [143]:
model.reactions.R_GALK2.gpr

G_WP_015926586_1

In [144]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015926586_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1427,B8I0X2,B8I0X2_RUMCH,Galactokinase (EC 2.7.1.6) (Galactose kinase),galK Ccel_3238,G_WP_015926586_1


In [145]:
model.reactions.R_GALKr

R_GALKr: M_atp_c + M_gal_c <-> M_adp_c + M_gal1p_c + M_h_c

In [146]:
model.reactions.R_GALKr.gpr

G_WP_015926586_1

In [144]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015926586_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1427,B8I0X2,B8I0X2_RUMCH,Galactokinase (EC 2.7.1.6) (Galactose kinase),galK Ccel_3238,G_WP_015926586_1


In [215]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015926586_1']:
    print(str(model.reactions[rxn]))#+": " +  str(model.reactions[rxn].gpr))

R_GALK2: M_a_gal__D_c + M_atp_c --> M_adp_c + M_gal1p_c + M_h_c
R_GALKr: M_atp_c + M_gal_c <-> M_adp_c + M_gal1p_c + M_h_c


### Glycolysis

**hexokinase**

In [11]:
model.reactions.R_HEX1

R_HEX1: M_atp_c + M_glc__D_c --> M_adp_c + M_g6p_c + M_h_c

In [15]:
model.reactions.R_HEX1.gpr.get_genes()

{'G_WP_015924247_1',
 'G_WP_015925130_1',
 'G_WP_015925627_1',
 'G_WP_015926569_1',
 'G_WP_015926770_1'}

In [171]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015924247_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1438,B8I7S2,B8I7S2_RUMCH,ROK family protein,Ccel_0700,G_WP_015924247_1


In [206]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015924247_1']:
    print(str(model.reactions[rxn]))#+": " +  str(model.reactions[rxn].gpr))

R_GLUKA_1: M_atp_c + M_glc__aD_c --> M_adp_c + M_g6p_A_c + M_h_c
R_HEX1: M_atp_c + M_glc__D_c --> M_adp_c + M_g6p_c + M_h_c


In [173]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925130_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
497,B8I2M1,B8I2M1_RUMCH,ROK family protein,Ccel_1663,G_WP_015925130_1


In [207]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015925130_1']:
    print(str(model.reactions[rxn]))#+": " +  str(model.reactions[rxn].gpr))

R_HEX1: M_atp_c + M_glc__D_c --> M_adp_c + M_g6p_c + M_h_c


In [175]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925627_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2628,B8I4M8,B8I4M8_RUMCH,ROK family protein,Ccel_2190,G_WP_015925627_1


In [208]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015925627_1']:
    print(str(model.reactions[rxn]))#+": " +  str(model.reactions[rxn].gpr))

R_AMANK: M_acmana_c + M_atp_c --> M_acmanap_c + M_adp_c + M_h_c
R_HEX1: M_atp_c + M_glc__D_c --> M_adp_c + M_g6p_c + M_h_c


In [177]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015926569_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1083,B8I0V5,B8I0V5_RUMCH,Hexokinase,Ccel_3221,G_WP_015926569_1


In [209]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015926569_1']:
    print(str(model.reactions[rxn]))#+": " +  str(model.reactions[rxn].gpr))

R_HEX1: M_atp_c + M_glc__D_c --> M_adp_c + M_g6p_c + M_h_c
R_HEX4: M_atp_c + M_man_c --> M_adp_c + M_h_c + M_man6p_c
R_HEX7: M_atp_c + M_fru_c --> M_adp_c + M_f6p_c + M_h_c


In [179]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015926770_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
259,B8I1T3,B8I1T3_RUMCH,ROK family protein,Ccel_3430,G_WP_015926770_1


In [180]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015926770_1']:
    print(str(model.reactions[rxn])+": " +  str(model.reactions[rxn].gpr))

R_HEX1: M_atp_c + M_glc__D_c --> M_adp_c + M_g6p_c + M_h_c: (G_WP_015924247_1 or G_WP_015925130_1 or G_WP_015925627_1 or G_WP_015926569_1 or G_WP_015926770_1)


In [72]:
model.reactions.R_GLUKA_1

R_GLUKA_1: M_atp_c + M_glc__aD_c --> M_adp_c + M_g6p_A_c + M_h_c

In [75]:
model.reactions.R_GLUKA_1.gpr

G_WP_015924247_1

**Phosphoglucomutase**

In [151]:
model.reactions.R_PGMT

R_PGMT: M_g1p_c <-> M_g6p_c

In [152]:
model.reactions.R_PGMT.gpr

G_WP_015925690_1

In [35]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925690_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2232,B8I4U4,B8I4U4_RUMCH,"2,3-bisphosphoglycerate-independent phosphogly...",gpmI Ccel_2259,G_WP_015925690_1


In [31]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925690_1']['Protein names'].values

array(['2,3-bisphosphoglycerate-independent phosphoglycerate mutase (BPG-independent PGAM) (Phosphoglyceromutase) (iPGM) (EC 5.4.2.12)'],
      dtype=object)

**Glucose-6-phosphate isomerase**

In [32]:
model.reactions.R_PGI

R_PGI: M_g6p_c <-> M_f6p_c

In [34]:
model.reactions.R_PGI.gpr

G_WP_015924943_1

In [36]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015924943_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2830,B8I1X3,B8I1X3_RUMCH,Glucose-6-phosphate isomerase (GPI) (EC 5.3.1....,pgi Ccel_1445,G_WP_015924943_1


In [77]:
model.reactions.R_G6PI.gpr

G_WP_015924943_1

**Phosphofructokinase**

In [68]:
model.reactions.R_PFK

R_PFK: M_atp_c + M_f6p_c --> M_adp_c + M_fdp_c + M_h_c

In [69]:
model.reactions.R_PFK.gpr

(G_WP_015925658_1 or G_WP_015926027_1)

In [91]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925658_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2707,B8I4R1,B8I4R1_RUMCH,Pyrophosphate--fructose 6-phosphate 1-phosphot...,pfp Ccel_2223,G_WP_015925658_1


In [92]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015926027_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
822,B8I6U8,B8I6U8_RUMCH,ATP-dependent 6-phosphofructokinase (ATP-PFK) ...,pfkA Ccel_2612,G_WP_015926027_1


In [216]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015925658_1']:
    print(str(model.reactions[rxn]) +": " +  str(model.reactions[rxn].gpr))

R_CD6P: M_ctp_c + M_f6p_c --> M_cdp_c + M_fdp_c + M_h_c: (G_WP_015925658_1 or G_WP_015926027_1)
R_CT6PT: M_ctp_c + M_tag6p__D_c --> M_cdp_c + M_h_c + M_tagdp__D_c: (G_WP_015925658_1 or G_WP_015926027_1)
R_ID6P: M_f6p_c + M_itp_c --> M_fdp_c + M_h_c + M_idp_c: (G_WP_015925658_1 or G_WP_015926027_1)
R_PFK: M_atp_c + M_f6p_c --> M_adp_c + M_fdp_c + M_h_c: (G_WP_015925658_1 or G_WP_015926027_1)
R_PFK_3: M_atp_c + M_s7p_c --> M_adp_c + M_h_c + M_s17bp_c: (G_WP_015925658_1 or G_WP_015926027_1)
R_UT6PT: M_tag6p__D_c + M_utp_c --> M_h_c + M_tagdp__D_c + M_udp_c: (G_WP_015925658_1 or G_WP_015926027_1)
R_r0191: M_f6p_c + M_utp_c --> M_fdp_c + M_h_c + M_udp_c: (G_WP_015925658_1 or G_WP_015926027_1)


In [210]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015926027_1']:
    print(str(model.reactions[rxn]))#+": " +  str(model.reactions[rxn].gpr))

R_CD6P: M_ctp_c + M_f6p_c --> M_cdp_c + M_fdp_c + M_h_c
R_CT6PT: M_ctp_c + M_tag6p__D_c --> M_cdp_c + M_h_c + M_tagdp__D_c
R_ID6P: M_f6p_c + M_itp_c --> M_fdp_c + M_h_c + M_idp_c
R_PFK: M_atp_c + M_f6p_c --> M_adp_c + M_fdp_c + M_h_c
R_PFK_2: M_atp_c + M_tag6p__D_c --> M_adp_c + M_h_c + M_tagdp__D_c
R_PFK_3: M_atp_c + M_s7p_c --> M_adp_c + M_h_c + M_s17bp_c
R_UT6PT: M_tag6p__D_c + M_utp_c --> M_h_c + M_tagdp__D_c + M_udp_c
R_r0191: M_f6p_c + M_utp_c --> M_fdp_c + M_h_c + M_udp_c


**Fructose-bisphosphate aldolase**

In [44]:
model.reactions.R_FBA

R_FBA: M_fdp_c <-> M_dhap_c + M_g3p_c

In [46]:
model.reactions.R_FBA.gpr

(G_WP_015925657_1 or G_WP_015925795_1)

In [47]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925657_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
467,B8I4R0,B8I4R0_RUMCH,"Fructose-1,6-bisphosphate aldolase, class II",Ccel_2222,G_WP_015925657_1


In [49]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925795_1']['Protein names'].values

array(['Deoxyribose-phosphate aldolase/phospho-2-dehydro-3-deoxyheptonate aldolase'],
      dtype=object)

In [86]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015925657_1']:
    print(str(model.reactions[rxn])+": " +  str(model.reactions[rxn].gpr))

R_FBA: M_fdp_c <-> M_dhap_c + M_g3p_c: (G_WP_015925657_1 or G_WP_015925795_1)
R_FBA2: M_f1p_c <-> M_dhap_c + M_glyald_c: (G_WP_015925657_1 or G_WP_015925795_1)
R_FBA3: M_s17bp_c <-> M_dhap_c + M_e4p_c: G_WP_015925657_1


**Glyceraldehyde-3-phosphate dehydrogenase**

In [59]:
model.reactions.R_GAPD

R_GAPD: M_g3p_c + M_nad_c + M_pi_c <-> M_13dpg_c + M_h_c + M_nadh_c

In [61]:
model.reactions.R_GAPD.gpr

G_WP_015925704_1

In [62]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925704_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1198,B8I4V8,B8I4V8_RUMCH,Glyceraldehyde-3-phosphate dehydrogenase (EC 1...,Ccel_2275,G_WP_015925704_1


In [85]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015925704_1']:
    print(str(model.reactions[rxn])+": " +  str(model.reactions[rxn].gpr))

R_E4PD: M_e4p_c + M_h2o_c + M_nad_c <-> M_4per_c + 2.0 M_h_c + M_nadh_c: G_WP_015925704_1
R_GAPD: M_g3p_c + M_nad_c + M_pi_c <-> M_13dpg_c + M_h_c + M_nadh_c: G_WP_015925704_1
R_NADHHR: M_h2o_c + M_nadh_c --> M_nadhx__R_c: (G_WP_015925704_1 or G_spontaneous)
R_NADHHS: M_h2o_c + M_nadh_c --> M_nadhx__S_c: (G_WP_015925704_1 or G_spontaneous)
R_NADPHHR: M_h2o_c + M_nadph_c --> M_nadphx__R_c: (G_WP_015925704_1 or G_spontaneous)
R_NADPHHS: M_h2o_c + M_nadph_c --> M_nadphx__S_c: (G_WP_015925704_1 or G_spontaneous)


**Phosphoglycerate kinase**

In [63]:
model.reactions.R_PGK

R_PGK: M_3pg_c + M_atp_c <-> M_13dpg_c + M_adp_c

In [65]:
model.reactions.R_PGK.gpr

G_WP_041706988_1

In [66]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_041706988_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)


**Phosphoglycerate mutase**

In [155]:
model.reactions.R_PGM

R_PGM: M_2pg_c <-> M_3pg_c

**Enolase**

In [158]:
model.reactions.R_ENO

R_ENO: M_2pg_c <-> M_h2o_c + M_pep_c

**Pyruvate kinase**

In [162]:
model.reactions.R_PYK

R_PYK: M_adp_c + M_h_c + M_pep_c --> M_atp_c + M_pyr_c

**Pyruvate-ferredoxin oxidoreductase**

In [167]:
model.reactions.R_POR_syn.name

'Pyruvate-ferredoxin oxidoreductase'

**Phosphoglycerate mutase**

In [89]:
model.reactions.R_PGM

R_PGM: M_2pg_c <-> M_3pg_c

In [102]:
model.reactions.R_PGM.gpr.get_genes()

{'G_WP_012634628_1',
 'G_WP_015924169_1',
 'G_WP_015925582_1',
 'G_WP_015925690_1'}

In [93]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_012634628_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
796,B8I4K9,B8I4K9_RUMCH,Metalloenzyme domain protein,Ccel_0175,G_WP_012634628_1


In [94]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015924169_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1101,B8I776,B8I776_RUMCH,Phosphoglycerate mutase,Ccel_0619,G_WP_015924169_1


In [95]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925582_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
1637,B8I451,B8I451_RUMCH,Proposed homoserine kinase,Ccel_2140,G_WP_015925582_1


In [103]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925690_1']['Protein names'].values

array(['2,3-bisphosphoglycerate-independent phosphoglycerate mutase (BPG-independent PGAM) (Phosphoglyceromutase) (iPGM) (EC 5.4.2.12)'],
      dtype=object)

In [97]:
for rxn in model.gene_to_reaction_lookup()['G_WP_012634628_1']:
    print(str(model.reactions[rxn])+": " +  str(model.reactions[rxn].gpr))

R_PGM: M_2pg_c <-> M_3pg_c: (G_WP_012634628_1 or G_WP_015924169_1 or G_WP_015925582_1 or G_WP_015925690_1)
R_PPM: M_r1p_c <-> M_r5p_c: (G_WP_012634628_1 or G_WP_015926553_1)
R_PPM2: M_2dr1p_c <-> M_2dr5p_c: (G_WP_012634628_1 or G_WP_015926553_1)


In [98]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015924169_1']:
    print(str(model.reactions[rxn])+": " +  str(model.reactions[rxn].gpr))

R_PGM: M_2pg_c <-> M_3pg_c: (G_WP_012634628_1 or G_WP_015924169_1 or G_WP_015925582_1 or G_WP_015925690_1)


In [99]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015925582_1']:
    print(str(model.reactions[rxn])+": " +  str(model.reactions[rxn].gpr))

R_PGM: M_2pg_c <-> M_3pg_c: (G_WP_012634628_1 or G_WP_015924169_1 or G_WP_015925582_1 or G_WP_015925690_1)


In [100]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015925690_1']:
    print(str(model.reactions[rxn])+": " +  str(model.reactions[rxn].gpr))

R_PGM: M_2pg_c <-> M_3pg_c: (G_WP_012634628_1 or G_WP_015924169_1 or G_WP_015925582_1 or G_WP_015925690_1)
R_PGMT: M_g1p_c <-> M_g6p_c: G_WP_015925690_1


In [101]:
model.reactions.R_PGMT.name

'Phosphoglucomutase'

### Fermentation related

**L-lactate dehydrogenase**

In [165]:
model.reactions.R_LDH_L

R_LDH_L: M_lac__L_c + M_nad_c <-> M_h_c + M_nadh_c + M_pyr_c

**Ferredoxin hydronase**

In [106]:
model.reactions.R_FNRR3

R_FNRR3: M_fdxo_2_2_c + M_h2_c --> M_fdxrd_c + 2.0 M_h_c

In [107]:
model.reactions.R_FNRR3.gpr

G_WP_015925729_1

In [108]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925729_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
387,B8I4Y3,B8I4Y3_RUMCH,Putative PAS/PAC sensor protein,Ccel_2300,G_WP_015925729_1


In [119]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015925729_1']:
    print(str(model.reactions[rxn])+": " +  str(model.reactions[rxn].gpr))

R_FNRR3: M_fdxo_2_2_c + M_h2_c --> M_fdxrd_c + 2.0 M_h_c: G_WP_015925729_1


**Acetaldehyde dehydrogenase (acetylating)**

In [183]:
model.reactions.R_ACALD

R_ACALD: M_acald_c + M_coa_c + M_nad_c <-> M_accoa_c + M_h_c + M_nadh_c

**Alcohol dehydrogenase (ethanol, NADP)**

In [116]:
model.reactions.R_ALCD2y

R_ALCD2y: M_etoh_c + M_nadp_c <-> M_acald_c + M_h_c + M_nadph_c

In [117]:
model.reactions.R_ALCD2y.gpr

G_WP_015924528_1

In [118]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015924528_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2909,B8I9B1,B8I9B1_RUMCH,Aldo/keto reductase,Ccel_1009,G_WP_015924528_1


In [212]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015924528_1']:
    print(str(model.reactions[rxn]))#+": " +  str(model.reactions[rxn].gpr))

R_ALCD2y: M_etoh_c + M_nadp_c <-> M_acald_c + M_h_c + M_nadph_c
R_ALR2: M_h_c + M_mthgxl_c + M_nadph_c --> M_acetol_c + M_nadp_c
R_ALR3: M_acetol_c + M_h_c + M_nadph_c --> M_12ppd__S_c + M_nadp_c
R_PPDOy: M_h_c + M_lald__D_c + M_nadph_c --> M_12ppd__R_c + M_nadp_c


**Alcohol dehydrogenase (ethanol)**

In [123]:
model.reactions.R_ALCD2x

R_ALCD2x: M_etoh_c + M_nad_c <-> M_acald_c + M_h_c + M_nadh_c

In [124]:
model.reactions.R_ALCD2x.gpr

G_WP_015926546_1

In [128]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015926546_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2983,B8I0G2,B8I0G2_RUMCH,Aldehyde-alcohol dehydrogenase,Ccel_3198,G_WP_015926546_1


In [213]:
for rxn in model.gene_to_reaction_lookup()['G_WP_015926546_1']:
    print(str(model.reactions[rxn]))#+": " +  str(model.reactions[rxn].gpr))

R_ACALD: M_acald_c + M_coa_c + M_nad_c <-> M_accoa_c + M_h_c + M_nadh_c
R_ALCD19: M_glyald_c + M_h_c + M_nadh_c <-> M_glyc_c + M_nad_c
R_ALCD19y: M_glyald_c + M_h_c + M_nadph_c <-> M_glyc_c + M_nadp_c
R_ALCD2x: M_etoh_c + M_nad_c <-> M_acald_c + M_h_c + M_nadh_c
R_BNOCA: M_btcoa_c + M_h_c + M_nadph_c --> M_btal_c + M_coa_c + M_nadp_c
R_BTCOARx: M_btcoa_c + M_h_c + M_nadh_c --> M_btal_c + M_coa_c + M_nad_c
R_BTS: M_btal_c + M_h_c + M_nadh_c <-> M_1btol_c + M_nad_c
R_BTS_nadph: M_btal_c + M_h_c + M_nadph_c --> M_1btol_c + M_nadp_c
R_CHOLD3: M_chol_c + M_nadp_c --> M_betald_c + M_h_c + M_nadph_c
R_IBHH: M_2mbald_c + M_nadh_c <-> M_h_c + M_ibtol_c + M_nad_c


**Phosphotransacetylase**

In [186]:
model.reactions.R_PTAr

R_PTAr: M_accoa_c + M_pi_c <-> M_actp_c + M_coa_c

**Acetate kinase**

In [189]:
model.reactions.R_ACKr

R_ACKr: M_ac_c + M_atp_c <-> M_actp_c + M_adp_c

### Other

**Phosphoenolpyruvate carboxykinase (GTP)**

In [192]:
model.reactions.R_PEPCK_re

R_PEPCK_re: M_gtp_c + M_oaa_c --> M_co2_c + M_gdp_c + M_pep_c

**Phosphoenolpyruvate carboxykinase**

In [195]:
model.reactions.R_PPCK

R_PPCK: M_atp_c + M_oaa_c --> M_adp_c + M_co2_c + M_pep_c

**Malate dehydrogenase**

In [198]:
model.reactions.R_MDH.name

'Malate dehydrogenase'

**Malic enzyme**

In [131]:
model.reactions.R_ME1

R_ME1: M_mal__L_c + M_nad_c --> M_co2_c + M_nadh_c + M_pyr_c

In [132]:
model.reactions.R_ME1.gpr

G_WP_012634592_1

In [133]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_012634592_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2784,B8I4H2,B8I4H2_RUMCH,Malic protein NAD-binding,Ccel_0138,G_WP_012634592_1


In [134]:
model.reactions.R_ME2

R_ME2: M_mal__L_c + M_nadp_c --> M_co2_c + M_nadph_c + M_pyr_c

In [135]:
model.reactions.R_ME2.gpr

G_WP_012634592_1

In [133]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_012634592_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
2784,B8I4H2,B8I4H2_RUMCH,Malic protein NAD-binding,Ccel_0138,G_WP_012634592_1


In [214]:
for rxn in model.gene_to_reaction_lookup()['G_WP_012634592_1']:
    print(str(model.reactions[rxn]))#+": " +  str(model.reactions[rxn].gpr))

R_ME1: M_mal__L_c + M_nad_c --> M_co2_c + M_nadh_c + M_pyr_c
R_ME2: M_mal__L_c + M_nadp_c --> M_co2_c + M_nadph_c + M_pyr_c
R_OAADC: M_h_c + M_oaa_c --> M_co2_c + M_pyr_c


In [137]:
gene_protein_map[gene_protein_map['Cross-reference (RefSeq)']=='G_WP_015925138_1']

,Entry,Entry name,Protein names,Gene names,Cross-reference (RefSeq)
3195,B8I2N0,B8I2N0_RUMCH,2-dehydro-3-deoxy-phosphogluconate aldolase (E...,Ccel_1672,G_WP_015925138_1


In [139]:
model.reactions.R_OAADC.name

'Oxaloacetate decarboxylase'

**Citrate synthase**

In [201]:
model.reactions.R_CS

R_CS: M_accoa_c + M_h2o_c + M_oaa_c --> M_cit_c + M_coa_c + M_h_c

**Aconitate hydratase**

In [204]:
model.reactions.R_ACONT

R_ACONT: M_cit_c <-> M_icit_c

**Isocitrate dehydrogenase**

In [205]:
model.reactions.R_ICDHyr

R_ICDHyr: M_icit_c + M_nadp_c <-> M_akg_c + M_co2_c + M_nadph_c